D'abord : se logguer à un compte gmail \
login : eleveenmi@gmail.com  (i=1,2,3,..) \
mwd : pwdeleveEnmi (i=1,2,3,..)

Avant de lancer le notebook, il faut avoir déplace le fichier learning.tar.gz du dossier partagé dans le dossier `stage_secondes_2025`.

#0) Installations, imports et fonctions

In [1]:
# Commencez par installer et faire les imports pour
 # avoir toutes les fonctions utiles
!pip install -q plotly
import os
import tarfile
import shutil
from google.colab import drive
from PIL import Image, ImageOps
import random
import torchvision.transforms as T
from matplotlib import pyplot as plt
import torchvision.transforms as T
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, balanced_accuracy_score
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, dataset
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import confusion_matrix
# Spécifique au TP :
! git clone https://github.com/nanopiero/intro-ia-secondes-2025.git
! cp intro-ia-secondes-2025/utils/apprentissage.py .
# from google.colab import files
# uploaded = files.upload()
from importlib import reload
import apprentissage
# reload(apprentissage)

from apprentissage import (
    show_confusion_matrix_plotly,
    get_valid_classes,
    clean_directory,
    apply_transforms,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    RandomDistortion,
    RandomCropping,
    Rotation,
    RandomColorJitter,
    show_filters,
    show_class_prediction_gallery_by_name,
    find_file_id
)

drive.mount('/content/drive')

Cloning into 'intro-ia-secondes-2025'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 243 (delta 69), reused 8 (delta 8), pack-reused 135 (from 1)
Receiving objects: 100% (243/243), 7.99 MiB | 17.94 MiB/s, done.
Resolving deltas: 100% (137/137), done.


# I) Chargeons le jeu d'entraînement

In [3]:
# D'abord, chargeons les images
tar_path = '/content/drive/MyDrive/stage_secondes_2025/learning.tar.gz'
extract_path = '/content/'

with tarfile.open(tar_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

# Ensuite précisons où sont les deux dossiers pour l'entraînelment et la validation
base_dir = os.path.join(extract_path, 'learning')
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In [ ]:
# Si dossiers vides :
valid_classes = get_valid_classes(train_dir, val_dir)
clean_directory(train_dir, valid_classes)
clean_directory(val_dir, valid_classes)

print("Nettoyage terminé. Nombre de classes restantes :", sorted(valid_classes))

#II) Choix d'une augmentation de données

In [ ]:
# Chargeons une image pour voir
image_path = "learning/train/21/" + os.listdir("learning/train/21")[0]
img = Image.open(image_path).convert("RGB")

# Definissons notre série de transformations
transforms = [
    RandomVerticalFlip(),
    RandomCropping(0.1),
    Rotation(10),
    RandomColorJitter(contrast=0.3, hue=0.05, saturation=0.3, brightness=0.3)
]



# Voyons l'image originale et sa transformée :
aug_img = apply_transforms(img.copy(), transforms)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Original")
plt.imshow(img)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Augmented")
plt.imshow(aug_img)
plt.axis("off")
plt.show()

#III) Entraînement d'un réseau de neurones

##III)1) Qu'est-ce qu'il y a dans un réseau de neurone (pour le traitement d'image) ?

In [ ]:
import torch
import torchvision.models as models
import matplotlib.pyplot as plt

# Chargeons un ResNet50 dont les poids sont choisis au hasard, et un Resnet pré-entraîné sur ImageNet
resnet_pretrained = models.resnet50(pretrained=True)
resnet_random = models.resnet50(pretrained=False)

# Get first conv layer weights: shape = [64, 3, 7, 7]
w_pre = resnet_pretrained.conv1.weight.data.clone()
w_rand = resnet_random.conv1.weight.data.clone()

show_filters(w_rand, "Random Init - First Conv Layer (R channel)")
show_filters(w_pre, "Pretrained on ImageNet - First Conv Layer (R channel)")
del resnet_pretrained, resnet_random

https://github.com/vdumoulin/conv_arithmetic

https://www.image-net.org/challenges/LSVRC/2012/index#task

## III)2) Comment entraîner un tel réseau ?

In [12]:
# ==== CONFIGURATION  ====
config = {
    'architecture': 'resnet18',   # 'resnet18' or 'resnet50'
    'freeze_backbone': False,
    'learning_rate': 0.001,
    'batch_size': 32,
    'num_epochs': 5,
    'image_size': 224,
    'train_dir': 'learning/train',
    'val_dir': 'learning/val',

    'transforms': [
        RandomVerticalFlip(),
        RandomCropping(0.1),
        Rotation(10),
        RandomColorJitter(contrast=0.3, hue=0.05, saturation=0.3, brightness=0.3)
    ]
}

In [13]:
# Construisons des "piocheurs" et des "chargeurs"
train_transform = T.Compose(config['transforms'] + [
    T.Resize((config['image_size'], config['image_size'])),
    T.ToTensor()
])

val_transform = T.Compose([
    T.Resize((config['image_size'], config['image_size'])),
    T.ToTensor()
])


train_set = ImageFolder(config['train_dir'], transform=train_transform)
val_set = ImageFolder(config['val_dir'], transform=val_transform)
class_names = train_set.classes

train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_set, batch_size=config['batch_size'], shuffle=False)


# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#Lançons l'apprentissage :

# Load model architecture
if config['architecture'] == 'resnet18':
    model = models.resnet18(pretrained=True)
elif config['architecture'] == 'resnet50':
    model = models.resnet50(pretrained=True)
else:
    raise ValueError("Unsupported architecture.")

# Freeze backbone if needed
if config['freeze_backbone']:
    for param in model.parameters():
        param.requires_grad = False

# Replace classifier
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

# Metric tracking
train_accs, val_accs = [], []
train_bal_accs, val_bal_accs = [], []

# === Training Loop ===
for epoch in range(config['num_epochs']):
    model.train()
    train_preds, train_labels = [], []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(train_labels, train_preds)
    train_bal = balanced_accuracy_score(train_labels, train_preds)
    train_accs.append(train_acc)
    train_bal_accs.append(train_bal)

    # Validation
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    val_bal = balanced_accuracy_score(val_labels, val_preds)
    val_accs.append(val_acc)
    val_bal_accs.append(val_bal)

    print(f"Epoch {epoch+1}/{config['num_epochs']}: "
          f"Train Acc: {train_acc:.3f}, Val Acc: {val_acc:.3f} | "
          f"Train BalAcc: {train_bal:.3f}, Val BalAcc: {val_bal:.3f}")


In [ ]:
#Voyons comment les scores ont progressé !

plt.figure(figsize=(10, 4))
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Val Accuracy')
plt.plot(train_bal_accs, label='Train Balanced Acc', linestyle='--')
plt.plot(val_bal_accs, label='Val Balanced Acc', linestyle='--')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training & Validation Metrics')
plt.legend()
plt.grid(True)
plt.show()

#IV) Visualisation des prédictions sur le jeu de validation

In [ ]:
# Voyons la table de confusion sur le jeu de validation :
show_confusion_matrix_plotly(model, val_loader, class_names, device)


In [ ]:
# Et quelques cas pris au hasard de bonnes/mauvaises prédictions :
show_class_prediction_gallery_by_name(model, val_loader, class_names, device, class_name='20')

#V) Sauvegarde du réseau et partage vers le compte central

In [ ]:
# Envoyez votre "meilleure" architecture pour comparaisons sur le jeu de test
groupe = 14

In [ ]:
# Save model weights
arch = config['architecture']
imgsize = config['image_size']
filename = f"{groupe}_{arch}_{imgsize}.pth"
torch.save(model.state_dict(), filename)

from google.colab import drive
import shutil
import os

# Mount drive if not already mounted
drive.mount('/content/drive')

drive_dir = "/content/drive/MyDrive/stage_secondes_2025"
os.makedirs(drive_dir, exist_ok=True)

# Move file
dst_path = os.path.join(drive_dir, filename)
shutil.move(filename, dst_path)
print(f"Saved to {dst_path}")


from google.colab import auth
from googleapiclient.discovery import build

auth.authenticate_user()
drive_service = build('drive', 'v3')

# Find file ID by name in destination folder
file_id = find_file_id(filename, drive_service)

# Share with email
drive_service.permissions().create(
    fileId=file_id,
    body={
        'type': 'user',
        'role': 'reader',
        'emailAddress': 'tpdeeplearning@gmail.com'
    },
    fields='id'
).execute()

print(f"Shared {filename} with tpdeeplearning@gmail.com")